In [12]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from datetime import datetime
import pandas_datareader as pdr
import pandas.util.testing as tm

In [13]:
SA_oil = pd.read_csv("2222.csv", usecols=['Date', 'Adj Close'],
                                   parse_dates=["Date"],
                                   index_col=["Date"])
#SA_oil = SA_oil.rename()
SA_oil.head()

,Adj Close
Date,
2020-01-01,28.788849
2020-02-01,28.114435
2020-03-01,25.416800
2020-04-01,26.749390
2020-05-01,27.890360


In [14]:
# retuen = (current - pervious) / current
returns_SA_oil = SA_oil.pct_change()
returns_SA_oil.head()
#returns_SA_oil.shape

,Adj Close
Date,
2020-01-01,NaN
2020-02-01,-0.023426
2020-03-01,-0.095952
2020-04-01,0.052429
2020-05-01,0.042654


In [15]:
Rajhi = pd.read_csv("1120.csv", usecols=['Date', 'Adj Close'],
                                   parse_dates=["Date"],
                                   index_col=["Date"])

#df.head()
returns_Rajhi = Rajhi.pct_change()
returns_Rajhi.head()

,Adj Close
Date,
2020-01-01,NaN
2020-02-01,-0.054217
2020-03-01,-0.143312
2020-04-01,0.093006
2020-05-01,0.008741


In [16]:
Sabic = df = pd.read_csv("2020.csv", usecols=['Date', 'Adj Close'],
                                   parse_dates=["Date"],
                                   index_col=["Date"])
#df.head()
returns_Sabic = Sabic.pct_change()
returns_Sabic.head()

,Adj Close
Date,
2020-01-01,NaN
2020-02-01,-0.104000
2020-03-01,-0.077381
2020-04-01,0.170016
2020-05-01,0.076271


In [17]:
#SA_oil, Rajhi, Sabic,
df = pd.merge(returns_SA_oil, returns_Rajhi, on='Date', how='inner')
#df


In [22]:
matrix_of_return_values = pd.merge(df, returns_Sabic, on='Date', how='inner')
matrix_of_return_values = matrix_of_return_value.dropna()
#matrix_of_return_value.index.duplicated().any()


In [8]:
#marge
#matrix_of_return_values = df1
#matrix_of_return_values.head().dropna()

In [23]:
collection_of_random_asset_weights = []
portfolio_means = []
portfolio_standard_deviations = []
portfolio_sharpe_ratios = []

In [44]:
#random_asset_weights = np.random.random(2)
#random_asset_weights /= random_asset_weights.sum()
#random_asset_weights
no_of_portfolios = 500
no_of_asset_classes = matrix_of_return_values.shape[1]

def get_random_asset_weights(no_of_asset_classes):
    random_asset_weights = np.random.rand(no_of_asset_classes)
    random_asset_weights /= sum(random_asset_weights)
    random_asset_weights = np.asmatrix(random_asset_weights)  # <-- Include this line.
    return random_asset_weights
asset_weights = get_random_asset_weights(no_of_asset_classes)   
asset_weights 
#print(type(no_of_asset_classes))

matrix([[0.324349  , 0.28659764, 0.38905336]])

In [42]:
def get_portfolio_statistics(asset_weights, matrix_of_return_values):
    asset_means = np.asmatrix(np.mean(matrix_of_return_values, axis = 0))
    #asset_covariances = np.asmatrix(np.cov(matrix_of_return_values))
    asset_covariances = np.asmatrix(matrix_of_return_values.cov())
    portfolio_mean = asset_weights * asset_means.T
    portfolio_mean = portfolio_mean.item()
    portfolio_standard_deviation = np.sqrt(asset_weights * asset_covariances * asset_weights.T)
    portfolio_standard_deviation = portfolio_standard_deviation.item()
    portfolio_sharpe_ratio = portfolio_mean / portfolio_standard_deviation
    return portfolio_mean, portfolio_standard_deviation, portfolio_sharpe_ratio  
    #print(portfolio_sharpe_ratio)
get_portfolio_statistics(asset_weights, matrix_of_return_values)    

(0.039713887014950926, 0.07137009117143386, 0.5564499969539979)

In [ ]:
while len(collection_of_random_asset_weights) < no_of_portfolios:
    random_asset_weights = asset_weights(no_of_asset_classes)
    collection_of_random_asset_weights.append(random_asset_weights) 
    A,B,C=get_portfolio_statistics(random_asset_weights,matrix_of_return_values)
    portfolio_means.append(A)
    portfolio_standard_deviations.append(B)
    portfolio_sharpe_ratios.append(C)